In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats
import config
from src import load_data
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go

In [2]:
data = load_data(config.PROCESSED_FILE).copy()


data["date"] = pd.to_datetime(data["date"])
data.set_index("date",inplace=True)

data.head()

Données chargées avec succès.


,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,Windspeed,Visibility,Tdewpoint,rv1,rv2,is_holiday,is_weekend,season,time_of_the_day,is_school_holiday
date,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,7.000000,63.000000,5.3,13.275433,13.275433,False,False,3,0.708333,False
2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,6.666667,59.166667,5.2,18.606195,18.606195,False,False,3,0.715278,False
2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,6.333333,55.333333,5.1,28.642668,28.642668,False,False,3,0.722222,False
2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,6.000000,51.500000,5.0,45.410389,45.410389,False,False,3,0.729167,False
2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,5.666667,47.666667,4.9,10.084097,10.084097,False,False,3,0.736111,False


In [3]:
def borne_min(array,valeur_borne):
    return (array>=valeur_borne)*array+(array<=valeur_borne)*valeur_borne

borne_min(np.array([1,0,2,5,7]),0.5)

array([1. , 0.5, 2. , 5. , 7. ])

In [4]:
t = np.arange(len(data))

X = pd.DataFrame({"Temps": t,"Sin_day": np.sin((2*np.pi*(t))/144),"intercept": np.ones((len(data),))})

variables_explicatives_list = ['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2', 'is_holiday',
        'is_school_holiday']

for j in range(24):
    X[f'is_{j+1}th_hour'] = (data.index.hour == j).astype(int)
        
for j in range(7):    
    X[f'is_{j+1}th_day'] = (data.index.day == j).astype(int)


X[variables_explicatives_list] = data[variables_explicatives_list].copy().to_numpy()

y = data["Appliances"].copy()

train_index = (data.index <= "2016-04-27 18:00:00").sum()


In [5]:
def mape(y_test,y_pred):
    return np.mean(np.abs((y_test-y_pred)/y_test))

def rmse(y_test,y_pred):
    return np.mean((y_test-y_pred)**2)**0.5


def plot_model(model="Linear",l1_ratio=0.5,alpha=1):
    if model not in ["Linear","Elastic_net"]:
        raise ValueError("If have to choose either 'Linear' or 'Elastic_net'")
    

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
    
    if model == "Linear":
        model1 = LinearRegression()
        model1.fit(X_train,y_train)

    elif model == "Elastic_net":
        model1 = ElasticNet(l1_ratio=l1_ratio,alpha=alpha)
        model1.fit(X_train,y_train)

    y_pred = borne_min(model1.predict(X_test),np.min(X_train))

    print(f"Le MAPE est {mape(y_test.to_numpy(),y_pred)}")
    print(f"Le RMSE est {rmse(y_test.to_numpy(),y_pred)}")

    if model == "Elastic_net":
        fig = go.Figure()

# Add the second line
        fig.add_trace(go.Scatter(x=data.index[:len(y_pred)], y=y_test, mode='lines', name='Test'))

# Add the first line
        fig.add_trace(go.Scatter(x=data.index[:len(y_pred)], y=y_pred, mode='lines', name='Prédiction'))

# Customize layout
        fig.update_layout(
        title=f"Prédiction {model}\n avec l1 = {l1_ratio} et alpha = {alpha}",
        xaxis_title="X-axis Label",
        yaxis_title="Y-axis Label",
        legend_title="Legend",
        template="plotly_white")

# Show the plot
        fig.show()


    else:
        fig = go.Figure()

# Add the second line
        fig.add_trace(go.Scatter(x=data.index[:len(y_pred)], y=y_test, mode='lines', name='Test'))

# Add the first line
        fig.add_trace(go.Scatter(x=data.index[:len(y_pred)], y=y_pred, mode='lines', name='Prédiction'))

# Customize layout
        fig.update_layout(
        title=f"Prédiction {model}",
        xaxis_title="X-axis Label",
        yaxis_title="Y-axis Label",
        legend_title="Legend",
        template="plotly_white")

# Show the plot
        fig.show()

        return y_pred    


In [6]:
def plot_online_model(model="Linear",l1_ratio=0.5,alpha=1,figure = True, arma_resid = False, order = (0,0,0), seasonnal_order = (0,0,0,0)):

    if model not in ["Linear","Elastic_net"]:
        raise ValueError("If have to choose either 'Linear' or 'Elastic_net'")

    step = 144 
    days = 1 
    total_steps = step * days
    
    
    y_pred = []
    
    for j in range(0, len(data) - train_index, total_steps):
        X_train, y_train = X.iloc[:train_index + j], y[:train_index + j]
        X_test = X.iloc[train_index + j:train_index + j + total_steps]
            
        model1 = ElasticNet(l1_ratio=l1_ratio, alpha=alpha) if model == "Elastic_net" else LinearRegression()
        model1.fit(X_train, y_train)

        y_train_pred = model1.predict(X_train)
        
        y_test_pred = model1.predict(X_test)

        if arma_resid:
            print('test')
            # Calculate residuals for the training set
            residuals = y_train - y_train_pred

            # Fit ARMA to the residuals
            arma_model = ARIMA(residuals, order=order, seasonal_order=seasonnal_order).fit()

            # Predict residuals for the test set
            residuals_pred = arma_model.forecast(steps=len(X_test))

            # Add predicted residuals to the main model's predictions
            y_test_pred += residuals_pred

        # Clip or adjust predictions as needed
        y_pred.append(borne_min(y_test_pred, np.min(y_train)))

    y_pred = np.hstack(y_pred)

    print(f"Le MAPE est {mape(y[train_index:train_index+len(y_pred)],y_pred)}")
    print(f"Le RMSE est {rmse(y_pred,y[train_index:train_index+len(y_pred)])}")

    if figure == True:
        fig = go.Figure()

# Add the second line
        fig.add_trace(go.Scatter(x=data.index[train_index:train_index+len(y_pred)], y=y[train_index:train_index+len(y_pred)], mode='lines', name='Test'))

# Add the first line
        fig.add_trace(go.Scatter(x=data.index[train_index:train_index+len(y_pred)], y=y_pred, mode='lines', name='Prédiction'))

# Customize layout
        fig.update_layout(
title=f"Prédiction {model}",
xaxis_title="X-axis Label",
yaxis_title="Y-axis Label",
legend_title="Legend",
template="plotly_white")

# Show the plot
        fig.show()  

    return y_pred

On cherche les contraintes l1 l2 optimale sur la base de données d'entrainements:

In [7]:
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV

train_index = (data.index <= "2016-04-27 18:00:00").sum()
X_train, y_train = X.iloc[:train_index], y[:train_index]
# Elastic Net model
model = ElasticNet(random_state=42)

# Hyperparameter grid
param_distributions = {
    "alpha": np.logspace(-4, 1, 50),  # Regularization strength
    "l1_ratio": np.linspace(0, 1, 50),  # Mix of L1 and L2 regularization
}

# Define TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Randomized search
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=125,  # Number of random combinations
    scoring="neg_mean_squared_error",  # Metric
    cv=tscv,  # Sequential cross-validation
    random_state=42,
    n_jobs=-1
)

# Fit the model
random_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", random_search.best_params_)

KeyboardInterrupt: 

In [ ]:
from itertools import product

# Grille pour (p, d, q)
p = range(0, 4)  # Ordre AR
d = range(0, 2)  # Ordre de différenciation
q = range(0, 4)  # Ordre MA
order_grid = list(product(p, d, q))

P = range(0, 2)
D = range(0, 2)
Q = range(0, 2)
S = [144]  # Périodicité saisonnière (par exemple, mensuelle)

seasonal_order_grid = list(product(P, D, Q, S))

def optimize_aic_sarima(residuals, order_grid, seasonal_order_grid):
    best_order = None
    best_seasonal_order = None
    best_aic = float("inf")

    for order in order_grid:
        for seasonal_order in seasonal_order_grid:
            try:
                # Ajuster le modèle SARIMA
                sarima_model = ARIMA(residuals, order=order, seasonal_order=seasonal_order).fit()
                aic = sarima_model.aic
                if aic < best_aic:
                    best_aic = aic
                    best_order = order
                    best_seasonal_order = seasonal_order
            except Exception as e:
                continue

    return best_order, best_seasonal_order, best_aic

# Calculer les résidus Elastic Net
residuals = y_train - random_search.best_estimator_.predict(X_train)

# Optimiser l'AIC
best_order, best_aic = optimize_aic_sarima(residuals, order_grid, seasonal_order_grid)

print("Best ARMA Order:", best_order)
print("Best AIC:", best_aic)


c:\Users\issam\OneDrive\Documents\cours\M2 ISDS\Apprentissage série temporel\appliances-energy-prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 10min will be used.
  self._init_dates(dates, freq)
c:\Users\issam\OneDrive\Documents\cours\M2 ISDS\Apprentissage série temporel\appliances-energy-prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 10min will be used.
  self._init_dates(dates, freq)
c:\Users\issam\OneDrive\Documents\cours\M2 ISDS\Apprentissage série temporel\appliances-energy-prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 10min will be used.
  self._init_dates(dates, freq)
c:\Users\issam\OneDrive\Documents\cours\M2 ISDS\Apprentissage série temporel\appliances-energy-prediction\venv\Lib\

In [8]:
plot_online_model(model="Elastic_net",l1_ratio=0.9591836734693877,alpha=0.3727593720314942)

Le MAPE est 0.4833337314823569
Le RMSE est 84.64444274602867


array([129.10536192, 162.12778531, 164.90311093, ..., 150.58647712,
       149.23200743, 158.87442385])

In [10]:
plot_online_model(model="Elastic_net",l1_ratio=0.9591836734693877,alpha=0.3727593720314942,figure = True, arma_resid = True, order = (1,1,1), seasonnal_order = (1,0,1,14))

test


c:\Users\issam\OneDrive\Documents\cours\M2 ISDS\Apprentissage série temporel\appliances-energy-prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 10min will be used.

c:\Users\issam\OneDrive\Documents\cours\M2 ISDS\Apprentissage série temporel\appliances-energy-prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 10min will be used.

c:\Users\issam\OneDrive\Documents\cours\M2 ISDS\Apprentissage série temporel\appliances-energy-prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 10min will be used.



test


c:\Users\issam\OneDrive\Documents\cours\M2 ISDS\Apprentissage série temporel\appliances-energy-prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 10min will be used.

c:\Users\issam\OneDrive\Documents\cours\M2 ISDS\Apprentissage série temporel\appliances-energy-prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 10min will be used.

c:\Users\issam\OneDrive\Documents\cours\M2 ISDS\Apprentissage série temporel\appliances-energy-prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 10min will be used.



test


...

c:\Users\issam\OneDrive\Documents\cours\M2 ISDS\Apprentissage série temporel\appliances-energy-prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 10min will be used.

c:\Users\issam\OneDrive\Documents\cours\M2 ISDS\Apprentissage série temporel\appliances-energy-prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 10min will be used.

c:\Users\issam\OneDrive\Documents\cours\M2 ISDS\Apprentissage série temporel\appliances-energy-prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 10min will be used.



test


c:\Users\issam\OneDrive\Documents\cours\M2 ISDS\Apprentissage série temporel\appliances-energy-prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 10min will be used.

c:\Users\issam\OneDrive\Documents\cours\M2 ISDS\Apprentissage série temporel\appliances-energy-prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 10min will be used.

c:\Users\issam\OneDrive\Documents\cours\M2 ISDS\Apprentissage série temporel\appliances-energy-prediction\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 10min will be used.



Le MAPE est 0.5019023827973612
Le RMSE est 84.95897202789041


array([294.89351741, 271.91707062, 237.21182119, ..., 156.19220445,
       154.85332166, 164.50512566])